One of my mentees did something really cool
https://colab.research.google.com/drive/12MTYkd6eZFbJ7tcuUxhNjStnwLdFunpL#scrollTo=6c219485

Here are some resources to look into
https://www.geeksforgeeks.org/python/pyyaml-fullloader-python/

https://medium.com/python-in-plain-english/using-yaml-file-as-a-small-db-in-python-68ebc61dd490
https://medium.com/@ThinkingLoop/10-pandas-data-cleaning-moves-that-saved-my-career-2fe4e8a50f18



These are the various steps we will take

1. Removing columns that are missing significantly data points  - creating a null report 📁 
2. Removing duplicates✅
2. Removing betting odds columns - this is like stealing✅
3. Confirm whether data columns types are correctly mapped
4. Canonicalize text (and stop death-by-typo) - cleaning text data with regular expressions, fixing cases
4. Normalize date formatsClean and create new date columns - year , month 

# Normalize date formats
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'])

5. One-hot and categorical the smart way - High-cardinality strings are memory hogs. Use category and selective encoding.
6. Automate validation checks using tools like Great Expectations.
7. Passing data to Neon Db - Using yaml files

import yaml
with open("configs/yaml" , "r") as f:
    data = yaml.safe_load(f)
username = data["production"]["username"]







Mindset shifts that changed everything
1. **Be explicit**. If a rule isn’t encoded in code, it’s a rumor.
2. **Prefer immutability**. Reassign results; avoid inplace=True for clarity and chaining.
3. **Choose honesty over prettiness**. A lower KPI with correct math beats a rosy one built on accidental exclusions.
4. **Automate small checks**. Five assertions can protect a million rows.

In [2]:
import pandas as pd
from pathlib import Path

# ===== Defining the data directories ====#
PROJECT_ROOT = Path().absolute().parent.parent
DATA_DIR = PROJECT_ROOT / "datasets" 
COMMON_DATA_DIR = DATA_DIR / "common_data"
INGESTED_DIR = DATA_DIR / "ingested_data"
ingested_csv_file = INGESTED_DIR / "enhanced_dataset.csv"

In [3]:
print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Common data directory: {COMMON_DATA_DIR}")
print(f"📁 Ingested data directory: {INGESTED_DIR}")

📁 Project root: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores
📁 Data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets
📁 Common data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\common_data
📁 Ingested data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\ingested_data


In [4]:
import pandas as pd
data = pd.read_csv(ingested_csv_file)
data.head()

C:\Users\juliu\AppData\Local\Temp\ipykernel_27504\3313027951.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(ingested_csv_file)


,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,SJA,GBH,GBD,GBA,BSH,BSD,BSA,SBH,SBD,SBA
0,16/08/2024,20:00,Man United,Fulham,1.0,0.0,H,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17/08/2024,12:30,Ipswich,Liverpool,0.0,2.0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17/08/2024,15:00,Arsenal,Wolves,2.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17/08/2024,15:00,Everton,Brighton,0.0,3.0,A,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17/08/2024,15:00,Newcastle,Southampton,1.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.shape

(6841, 166)

In [6]:
data.columns

Index(['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR',
       ...
       'SJA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'SBH', 'SBD', 'SBA'],
      dtype='object', length=166)

**Initial Deletion of columns - columns that are mostly empty**

In [7]:
print(f"The number of columns are ",data.shape[1], " and the number of the rows are",data.shape[0] )

The number of columns are  166  and the number of the rows are 6841


In [8]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1, keys=['Total','Percent'])
missing_data.head(20)

,Total,Percent
1XBD,6470,0.945768
1XBH,6470,0.945768
1XBA,6470,0.945768
BFE<2.5,6464,0.944891
BFE>2.5,6464,0.944891
BFA,6462,0.944599
BFH,6462,0.944599
BFD,6462,0.944599
BFCA,6461,0.944453
BFCH,6461,0.944453


In [9]:
new_data = data.drop(missing_data[missing_data['Percent'] > 0.10].index, axis=1)
new_data.isnull().sum()
new_data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,WHD,WHA,season_id,competition_name,IWH,IWD,IWA,VCH,VCD,VCA
0,16/08/2024,Man United,Fulham,1.0,0.0,H,0.0,0.0,D,R Jones,...,4.2,5.00,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
1,17/08/2024,Ipswich,Liverpool,0.0,2.0,A,0.0,0.0,D,T Robinson,...,5.5,1.33,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
2,17/08/2024,Arsenal,Wolves,2.0,0.0,H,1.0,0.0,H,J Gillett,...,7.0,17.00,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
3,17/08/2024,Everton,Brighton,0.0,3.0,A,0.0,1.0,A,S Hooper,...,3.5,2.70,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
4,17/08/2024,Newcastle,Southampton,1.0,0.0,H,1.0,0.0,H,C Pawson,...,5.5,8.00,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
new_data.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA',
       'WHH', 'WHD', 'WHA', 'season_id', 'competition_name', 'IWH', 'IWD',
       'IWA', 'VCH', 'VCD', 'VCA'],
      dtype='object')

In [11]:
print(f"The number of columns are ",new_data.shape[1], " and the number of the rows are",new_data.shape[0] )

The number of columns are  39  and the number of the rows are 6841


In [12]:
total = new_data.isnull().sum().sort_values(ascending=False)
percent = (new_data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_rows = new_data.shape[0]


missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
IWH,565.0,0.082590
IWD,565.0,0.082590
IWA,565.0,0.082590
VCH,381.0,0.055694
VCD,381.0,0.055694
VCA,381.0,0.055694
WHA,199.0,0.029089
WHH,199.0,0.029089
WHD,199.0,0.029089
BWD,145.0,0.021196


In [13]:
print(f"The number of columns are ",new_data.shape[1], " and the number of the rows are",new_data.shape[0] )

The number of columns are  39  and the number of the rows are 6841


In [14]:
new_data.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,WHD,WHA,season_id,competition_name,IWH,IWD,IWA,VCH,VCD,VCA
0,16/08/2024,Man United,Fulham,1.0,0.0,H,0.0,0.0,D,R Jones,...,4.20,5.00,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
1,17/08/2024,Ipswich,Liverpool,0.0,2.0,A,0.0,0.0,D,T Robinson,...,5.50,1.33,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
2,17/08/2024,Arsenal,Wolves,2.0,0.0,H,1.0,0.0,H,J Gillett,...,7.00,17.00,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
3,17/08/2024,Everton,Brighton,0.0,3.0,A,0.0,1.0,A,S Hooper,...,3.50,2.70,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
4,17/08/2024,Newcastle,Southampton,1.0,0.0,H,1.0,0.0,H,C Pawson,...,5.50,8.00,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
5,17/08/2024,Nott'm Forest,Bournemouth,1.0,1.0,D,1.0,0.0,H,M Oliver,...,3.50,2.80,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
6,17/08/2024,West Ham,Aston Villa,1.0,2.0,A,1.0,1.0,D,T Harrington,...,3.75,2.75,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
7,18/08/2024,Brentford,Crystal Palace,2.0,1.0,H,1.0,0.0,H,S Barrott,...,3.50,2.88,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
8,18/08/2024,Chelsea,Man City,0.0,2.0,A,0.0,1.0,A,A Taylor,...,4.00,1.83,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN
9,19/08/2024,Leicester,Tottenham,1.0,1.0,D,0.0,1.0,A,C Kavanagh,...,4.33,1.65,2024-2025,EPL,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
new_data.sample(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,WHD,WHA,season_id,competition_name,IWH,IWD,IWA,VCH,VCD,VCA
1359,09/02/2022,Norwich,Crystal Palace,1.0,1.0,D,1.0,0.0,H,P Tierney,...,3.30,2.15,2021-2022,EPL,3.45,3.35,2.15,3.50,3.20,2.15
4907,29/04/13,Aston Villa,Sunderland,6.0,1.0,H,2.0,1.0,H,L Probert,...,3.20,3.20,2012-2013,EPL,2.00,3.20,3.70,2.30,3.40,3.40
6253,20/12/08,Hull,Sunderland,1.0,4.0,A,1.0,1.0,D,M Riley,...,3.20,3.60,2008-2009,EPL,2.10,3.10,3.20,2.00,3.30,3.80
2755,28/10/2017,Watford,Stoke,0.0,1.0,A,0.0,1.0,A,M Oliver,...,3.50,3.75,2017-2018,EPL,2.10,3.40,3.50,2.05,3.60,3.90
5990,14/03/10,Sunderland,Man City,1.0,1.0,D,1.0,0.0,H,C Foy,...,3.25,2.15,2009-2010,EPL,3.60,3.30,2.00,3.60,3.25,2.10
3294,25/02/17,Watford,West Ham,1.0,1.0,D,1.0,0.0,H,C Pawson,...,3.20,2.90,2016-2017,EPL,2.60,3.25,2.75,2.63,3.30,2.90
3613,02/01/2016,Norwich,Southampton,1.0,0.0,H,0.0,0.0,D,M Clattenburg,...,3.20,2.38,2015-2016,EPL,2.90,3.30,2.30,3.25,3.40,2.38
4427,08/02/14,Sunderland,Hull,0.0,2.0,A,0.0,1.0,A,M Jones,...,3.00,3.75,2013-2014,EPL,1.95,3.35,3.70,2.15,3.30,3.90
3602,28/12/2015,Everton,Stoke,3.0,4.0,A,1.0,2.0,A,M Clattenburg,...,3.50,3.80,2015-2016,EPL,1.90,3.45,3.80,1.95,3.60,4.33
3692,01/03/2016,Sunderland,Crystal Palace,2.0,2.0,D,1.0,0.0,H,A Taylor,...,3.10,3.00,2015-2016,EPL,2.50,3.30,2.65,2.50,3.30,3.13


In [16]:
new_data.sample(frac=0.002)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,WHD,WHA,season_id,competition_name,IWH,IWD,IWA,VCH,VCD,VCA
3062,27/08/16,Everton,Stoke,1.0,0.0,H,0.0,0.0,D,M Oliver,...,3.6,4.75,2016-2017,EPL,1.70,3.60,4.70,1.73,3.80,5.50
2174,29/02/2020,West Ham,Southampton,3.0,1.0,H,2.0,1.0,H,A Taylor,...,3.5,2.55,2019-2020,EPL,2.70,3.50,2.55,2.75,3.50,2.60
3053,20/08/16,Stoke,Man City,1.0,4.0,A,0.0,2.0,A,M Dean,...,3.5,1.73,2016-2017,EPL,4.70,3.60,1.70,5.40,3.90,1.73
6347,01/03/09,West Ham,Man City,1.0,0.0,H,0.0,0.0,D,M Dean,...,3.3,2.90,2008-2009,EPL,2.30,3.30,2.90,2.50,3.20,2.75
747,14/05/2024,Tottenham,Man City,0.0,2.0,A,0.0,0.0,D,C Kavanagh,...,5.5,1.40,2023-2024,EPL,NaN,NaN,NaN,7.50,5.50,1.36
1248,07/11/2021,Leeds,Leicester,1.0,1.0,D,1.0,1.0,D,D England,...,3.5,2.45,2021-2022,EPL,2.75,3.55,2.50,2.70,3.50,2.40
5274,15/04/12,Man United,Aston Villa,4.0,0.0,H,2.0,0.0,H,M Halsey,...,5.5,12.00,2011-2012,EPL,1.25,5.00,10.00,1.22,6.50,15.00
3611,02/01/2016,Leicester,Bournemouth,0.0,0.0,D,0.0,0.0,D,A Marriner,...,3.4,4.00,2015-2016,EPL,1.90,3.45,3.80,1.93,3.75,4.20
3964,20/12/14,Southampton,Everton,3.0,0.0,H,1.0,0.0,H,J Moss,...,3.2,2.80,2014-2015,EPL,2.30,3.30,2.90,2.45,3.30,3.10
5501,28/12/10,Birmingham,Man United,1.0,1.0,D,0.0,0.0,D,L Mason,...,3.8,1.53,2010-2011,EPL,5.40,3.70,1.55,7.50,4.20,1.50


In [17]:
print(new_data.sample(10))


            Date      HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
4772    12/01/13       Norwich      Newcastle   0.0   0.0   D   0.0   0.0   D   
1558  17/10/2020       Everton      Liverpool   2.0   2.0   D   1.0   1.0   D   
6208    16/11/08       Everton  Middlesbrough   1.0   1.0   D   0.0   1.0   A   
2852  26/12/2017  Huddersfield          Stoke   1.0   1.0   D   1.0   0.0   H   
286   16/03/2025       Arsenal        Chelsea   1.0   0.0   H   1.0   0.0   H   
2733  14/10/2017      Man City          Stoke   7.0   2.0   H   3.0   1.0   H   
2636  27/04/2019       Watford         Wolves   1.0   2.0   A   0.0   1.0   A   
250   19/02/2025   Aston Villa      Liverpool   2.0   2.0   D   2.0   1.0   H   
3326    01/04/17     Leicester          Stoke   2.0   0.0   H   1.0   0.0   H   
3462  12/09/2015       Everton        Chelsea   3.0   1.0   H   2.0   1.0   H   

         Referee  ...   WHD    WHA  season_id  competition_name   IWH   IWD  \
4772    A Taylor  ...  3.25  

In [18]:
null_report = (
    new_data.isna()
      .mean()
      .rename("missing_rate")
      .to_frame()
      .assign(n_null=new_data.isna().sum())
      .sort_values("missing_rate", ascending=False)
)
null_report

,missing_rate,n_null
IWH,0.082590,565
IWD,0.082590,565
IWA,0.082590,565
VCH,0.055694,381
VCD,0.055694,381
VCA,0.055694,381
WHA,0.029089,199
WHH,0.029089,199
WHD,0.029089,199
BWD,0.021196,145


### Duplicate rows removal


In [19]:
#identify duplicate rows
duplicateRows = new_data[new_data.duplicated()]

#view duplicate rows
duplicateRows

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,WHD,WHA,season_id,competition_name,IWH,IWD,IWA,VCH,VCD,VCA


### Incorrect data types

In [20]:
new_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841 entries, 0 to 6840
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              6840 non-null   object 
 1   HomeTeam          6840 non-null   object 
 2   AwayTeam          6840 non-null   object 
 3   FTHG              6840 non-null   float64
 4   FTAG              6840 non-null   float64
 5   FTR               6840 non-null   object 
 6   HTHG              6840 non-null   float64
 7   HTAG              6840 non-null   float64
 8   HTR               6840 non-null   object 
 9   Referee           6840 non-null   object 
 10  HS                6840 non-null   float64
 11  AS                6840 non-null   float64
 12  HST               6840 non-null   float64
 13  AST               6840 non-null   float64
 14  HF                6840 non-null   float64
 15  AF                6840 non-null   float64
 16  HC                6840 non-null   float64


In [21]:
new_df = new_data.dropna()

print(new_df.to_string())

            Date          HomeTeam          AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR        Referee    HS    AS   HST   AST    HF    AF    HC    AC   HY   AY   HR   AR  B365H  B365D  B365A    BWH    BWD    BWA    WHH    WHD    WHA  season_id competition_name    IWH    IWD    IWA     VCH    VCD     VCA
380   11/08/2023           Burnley          Man City   0.0   3.0   A   0.0   2.0   A       C Pawson   6.0  17.0   1.0   8.0  11.0   8.0   6.0   5.0  0.0  0.0  1.0  0.0   8.00   5.50   1.33   8.75   5.25   1.34   8.00   5.00   1.25  2023-2024              EPL   8.00   5.50   1.35   9.500   5.25   1.330
381   12/08/2023           Arsenal     Nott'm Forest   2.0   1.0   H   2.0   0.0   H       M Oliver  15.0   6.0   7.0   2.0  12.0  12.0   8.0   3.0  2.0  2.0  0.0  0.0   1.18   7.00  15.00   1.17   7.50  15.50   1.12   6.50  12.00  2023-2024              EPL   1.20   7.25  14.00   1.140   7.50  17.000
382   12/08/2023       Bournemouth          West Ham   1.0   1.0   D   0.0   0.0   D       

In [22]:
new_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841 entries, 0 to 6840
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              6840 non-null   object 
 1   HomeTeam          6840 non-null   object 
 2   AwayTeam          6840 non-null   object 
 3   FTHG              6840 non-null   float64
 4   FTAG              6840 non-null   float64
 5   FTR               6840 non-null   object 
 6   HTHG              6840 non-null   float64
 7   HTAG              6840 non-null   float64
 8   HTR               6840 non-null   object 
 9   Referee           6840 non-null   object 
 10  HS                6840 non-null   float64
 11  AS                6840 non-null   float64
 12  HST               6840 non-null   float64
 13  AST               6840 non-null   float64
 14  HF                6840 non-null   float64
 15  AF                6840 non-null   float64
 16  HC                6840 non-null   float64


In [23]:
def inc(x):
    return x + 1


def test_answer():
    assert inc(4) == 5

test_answer()

In [24]:
#Standardize column names: lowercase, snake_case, strip spaces

#Borrowed from - Phoebe


#cohort3_DS_clean = cohort3_DS.copy()
#cohort3_DS_clean.columns = (
   # cohort3_DS_clean.columns
  #  .str.strip()
  #  .str.lower()
   # .str.replace(" ", "_")
    #.str.replace("?", "", regex=False)
#)
#cohort3_DS_clean.head()

# Data Cleaning (begins here)

##  1. Removing Betting Columns

In [25]:

import pandas as pd
from pathlib import Path

# ===== Defining the data directories ====#
PROJECT_ROOT = Path().absolute().parent.parent
DATA_DIR = PROJECT_ROOT / "datasets" 
COMMON_DATA_DIR = DATA_DIR / "common_data"
INGESTED_DIR = DATA_DIR / "ingested_data"

# == Ingested csv file == #
ingested_csv_file = INGESTED_DIR / "enhanced_dataset.csv"

# == Feature catalog csv == #
feature_catalog_csv_file = COMMON_DATA_DIR / "feature_catalog.csv"


print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Common data directory: {COMMON_DATA_DIR}")
print(f"📁 Ingested data directory: {INGESTED_DIR}")

📁 Project root: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores
📁 Data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets
📁 Common data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\common_data
📁 Ingested data directory: c:\Users\juliu\OneDrive\Desktop\Projects\Soca-Scores\datasets\ingested_data


In [26]:
# Enhanced dataset
enhanced_data = pd.read_csv(ingested_csv_file)
enhanced_data.sample(3)


C:\Users\juliu\AppData\Local\Temp\ipykernel_27504\670297540.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  enhanced_data = pd.read_csv(ingested_csv_file)


,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,SJA,GBH,GBD,GBA,BSH,BSD,BSA,SBH,SBD,SBA
2089,28/12/2019,12:30,Brighton,Bournemouth,2.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5065,26/11/11,NaN,Stoke,Blackburn,3.0,1.0,H,1.0,0.0,H,...,4.75,1.8,3.4,4.5,1.75,3.4,4.75,1.75,3.5,4.6
2249,13/07/2020,20:00,Man United,Southampton,2.0,2.0,D,2.0,1.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
enhanced_data.dtypes

Date         object
Time         object
HomeTeam     object
AwayTeam     object
FTHG        float64
             ...   
BSD         float64
BSA         float64
SBH         float64
SBD         float64
SBA         float64
Length: 166, dtype: object

In [28]:
# === Removing betting odds features from the dataset === #
# Feature columns dataset
feature_catalog = pd.read_csv(feature_catalog_csv_file)
feature_catalog.sample(3)

,Feature_name,Feature_description,Betting_odd
84,VCD,"VC Bet draw odds (now BetVictor, see above)",True
114,P>2.5,Pinnacle over 2.5 goals,True
38,BFH,Betfair home win odds,True


In [29]:
enhanced_data.shape

(6841, 166)

In [30]:
# === Using MultiIndex with isin() === #

filtered_feature_catalog = feature_catalog[feature_catalog["Betting_odd"]==False].copy()
filtered_feature_catalog.sample(2)



,Feature_name,Feature_description,Betting_odd
29,AR,Away Team Red Cards,False
7,FTR,"Full Time Result (H=Home Win, D=Draw, A=Away Win)",False


In [31]:

# 1. Get the column names to keep as a set for fast lookup
feature_names_to_keep = filtered_feature_catalog['Feature_name'].unique()
enhanced_data = enhanced_data.loc[:, enhanced_data.columns.isin(feature_names_to_keep)].copy()
enhanced_data

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,16/08/2024,20:00,Man United,Fulham,1.0,0.0,H,0.0,0.0,D,...,5.0,2.0,12.0,10.0,7.0,8.0,2.0,3.0,0.0,0.0
1,17/08/2024,12:30,Ipswich,Liverpool,0.0,2.0,A,0.0,0.0,D,...,2.0,5.0,9.0,18.0,2.0,10.0,3.0,1.0,0.0,0.0
2,17/08/2024,15:00,Arsenal,Wolves,2.0,0.0,H,1.0,0.0,H,...,6.0,3.0,17.0,14.0,8.0,2.0,2.0,2.0,0.0,0.0
3,17/08/2024,15:00,Everton,Brighton,0.0,3.0,A,0.0,1.0,A,...,1.0,5.0,8.0,8.0,1.0,5.0,1.0,1.0,1.0,0.0
4,17/08/2024,15:00,Newcastle,Southampton,1.0,0.0,H,1.0,0.0,H,...,1.0,4.0,15.0,16.0,3.0,12.0,2.0,4.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6836,11/05/08,NaN,Portsmouth,Fulham,0.0,1.0,A,0.0,0.0,D,...,10.0,4.0,13.0,11.0,5.0,6.0,0.0,0.0,0.0,0.0
6837,11/05/08,NaN,Sunderland,Arsenal,0.0,1.0,A,0.0,1.0,A,...,5.0,8.0,14.0,12.0,3.0,4.0,1.0,1.0,0.0,0.0
6838,11/05/08,NaN,Tottenham,Liverpool,0.0,2.0,A,0.0,0.0,D,...,3.0,12.0,11.0,9.0,8.0,4.0,1.0,1.0,0.0,0.0
6839,11/05/08,NaN,West Ham,Aston Villa,2.0,2.0,D,1.0,1.0,D,...,8.0,8.0,9.0,21.0,7.0,8.0,2.0,2.0,0.0,0.0


##  2. Removing columns missing a significant chunk of data  - creating null report

In [32]:
print(f"The number of columns are ",enhanced_data.shape[1], " and the number of the rows are",enhanced_data.shape[0] )

The number of columns are  23  and the number of the rows are 6841


The percent series is the result of a division, but when you concatenate it with the total series, the resulting 'Percent' column in missing_data might be stored as an object type (or sometimes a type that prevents direct numeric comparison)

total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1, keys=['Total','Percent'])
missing_data.head(20)

In [33]:
total = enhanced_data.isnull().sum().sort_values(ascending=False)
missing_percent = (enhanced_data.isnull().sum() / len(enhanced_data)) * 100
missing_data = pd.concat([total,missing_percent],axis=1, keys=['Total','Percent'])
missing_data.head(20)

,Total,Percent
Time,4561,66.671539
Date,1,0.014618
HomeTeam,1,0.014618
AwayTeam,1,0.014618
FTHG,1,0.014618
FTAG,1,0.014618
FTR,1,0.014618
HTHG,1,0.014618
HTAG,1,0.014618
HTR,1,0.014618


In [34]:
#missing_percent = (enhanced_data.isnull().sum() / len(enhanced_data)) * 100

# 2. Identify the column names (the Series index) where the percentage is > 10
columns_to_drop = missing_percent[missing_percent > 10].index

# 3. Drop all identified columns in a single operation
enhanced_data_with_no_nulls = enhanced_data.drop(columns=columns_to_drop).copy()

# The .copy() is used here at the end to ensure the result is an independent DataFrame.
print(f"Dropped {len(columns_to_drop)} columns.")
print(f"Dropped {columns_to_drop} columns.")
print(enhanced_data_with_no_nulls.head())

Dropped 1 columns.
Dropped Index(['Time'], dtype='object') columns.
         Date    HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0  16/08/2024  Man United       Fulham   1.0   0.0   H   0.0   0.0   D   
1  17/08/2024     Ipswich    Liverpool   0.0   2.0   A   0.0   0.0   D   
2  17/08/2024     Arsenal       Wolves   2.0   0.0   H   1.0   0.0   H   
3  17/08/2024     Everton     Brighton   0.0   3.0   A   0.0   1.0   A   
4  17/08/2024   Newcastle  Southampton   1.0   0.0   H   1.0   0.0   H   

      Referee  ...  HST  AST    HF    AF   HC    AC   HY   AY   HR   AR  
0     R Jones  ...  5.0  2.0  12.0  10.0  7.0   8.0  2.0  3.0  0.0  0.0  
1  T Robinson  ...  2.0  5.0   9.0  18.0  2.0  10.0  3.0  1.0  0.0  0.0  
2   J Gillett  ...  6.0  3.0  17.0  14.0  8.0   2.0  2.0  2.0  0.0  0.0  
3    S Hooper  ...  1.0  5.0   8.0   8.0  1.0   5.0  1.0  1.0  1.0  0.0  
4    C Pawson  ...  1.0  4.0  15.0  16.0  3.0  12.0  2.0  4.0  1.0  0.0  

[5 rows x 22 columns]


In [35]:
columns_to_drop = missing_data[missing_data['Percent'] > 10].index.tolist()
enhanced_data_with_no_nulls = enhanced_data.drop(columns=columns_to_drop)
enhanced_data_with_no_nulls.head()
enhanced_data.shape

(6841, 23)

## 3. Declare Data Types

Messy CSVs quietly produce object columns for booleans, numerics, and dates. Everything slows down, comparisons misfire, and merges break.

In [36]:
enhanced_data_with_no_nulls.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR'],
      dtype='object')

In [37]:
enhanced_data_with_no_nulls.dtypes

Date         object
HomeTeam     object
AwayTeam     object
FTHG        float64
FTAG        float64
FTR          object
HTHG        float64
HTAG        float64
HTR          object
Referee      object
HS          float64
AS          float64
HST         float64
AST         float64
HF          float64
AF          float64
HC          float64
AC          float64
HY          float64
AY          float64
HR          float64
AR          float64
dtype: object

In [38]:
# Convert data dtypes to actual types
enhanced_data_with_no_nulls.sample(3)


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
20,31/08/2024,Arsenal,Brighton,1.0,1.0,D,1.0,0.0,H,C Kavanagh,...,7.0,4.0,12.0,7.0,3.0,7.0,3.0,2.0,1.0,0.0
1784,04/03/2021,Liverpool,Chelsea,0.0,1.0,A,0.0,1.0,A,M Atkinson,...,1.0,5.0,9.0,8.0,5.0,2.0,0.0,0.0,0.0,0.0
2319,02/09/2018,Watford,Tottenham,2.0,1.0,H,0.0,0.0,D,A Marriner,...,3.0,2.0,9.0,8.0,3.0,10.0,2.0,1.0,0.0,0.0


In [39]:
# == Fixing Date Column x Augmenting "Date" Column == #

enhanced_data_with_no_nulls['Date'] = pd.to_datetime(enhanced_data_with_no_nulls['Date'],format = 'mixed')
enhanced_data_with_no_nulls['Month'] = enhanced_data_with_no_nulls['Date'].dt.strftime('%B')
enhanced_data_with_no_nulls['Year'] = enhanced_data_with_no_nulls['Date'].dt.year
enhanced_data_with_no_nulls['Day'] = enhanced_data_with_no_nulls['Date'].dt.strftime('%A')
enhanced_data_with_no_nulls.sample(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Month,Year,Day
6734,2008-01-03,Middlesbrough,Reading,0.0,1.0,A,0.0,0.0,D,H Webb,...,17.0,7.0,2.0,4.0,5.0,0.0,0.0,January,2008.0,Thursday
324,2025-04-19,Aston Villa,Newcastle,4.0,1.0,H,1.0,1.0,D,J Gillett,...,12.0,7.0,7.0,1.0,3.0,0.0,0.0,April,2025.0,Saturday
4902,2013-04-27,Stoke,Norwich,1.0,0.0,H,0.0,0.0,D,A Taylor,...,8.0,5.0,3.0,4.0,2.0,0.0,0.0,April,2013.0,Saturday
4521,2014-04-19,Newcastle,Swansea,1.0,2.0,A,1.0,1.0,D,C Foy,...,9.0,2.0,8.0,2.0,0.0,0.0,0.0,April,2014.0,Saturday
4066,2015-01-03,Arsenal,Everton,2.0,0.0,H,1.0,0.0,H,A Marriner,...,5.0,8.0,9.0,2.0,0.0,0.0,0.0,January,2015.0,Saturday
4663,2012-10-11,Reading,Norwich,0.0,0.0,D,0.0,0.0,D,C Foy,...,16.0,10.0,3.0,0.0,1.0,0.0,0.0,October,2012.0,Thursday
2410,2018-11-30,Cardiff,Wolves,2.0,1.0,H,0.0,1.0,A,A Marriner,...,12.0,7.0,6.0,1.0,2.0,0.0,0.0,November,2018.0,Friday
962,2023-04-02,Brentford,Southampton,3.0,0.0,H,2.0,0.0,H,D Bond,...,9.0,4.0,4.0,1.0,2.0,0.0,0.0,April,2023.0,Sunday
3511,2015-10-24,Aston Villa,Swansea,1.0,2.0,A,0.0,0.0,D,N Swarbrick,...,8.0,4.0,6.0,2.0,3.0,0.0,0.0,October,2015.0,Saturday
369,2025-05-20,Man City,Bournemouth,3.0,1.0,H,2.0,0.0,H,T Bramall,...,13.0,3.0,1.0,1.0,3.0,1.0,1.0,May,2025.0,Tuesday


In [40]:
#  == Fixing String Columns == #


string_cols = ['HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'Referee', 'Month', 'Day'] 

# 2. Map columns to the 'str' type
str_map = ({col: str for col in string_cols})

# 3. Apply the conversion
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls.astype(str_map) 
enhanced_data_with_no_nulls.dtypes

Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG               float64
FTAG               float64
FTR                 object
HTHG               float64
HTAG               float64
HTR                 object
Referee             object
HS                 float64
AS                 float64
HST                float64
AST                float64
HF                 float64
AF                 float64
HC                 float64
AC                 float64
HY                 float64
AY                 float64
HR                 float64
AR                 float64
Month               object
Year               float64
Day                 object
dtype: object

In [41]:
# == Fixing Numeric Columns == #

num_cols = ['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'HS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR','Year']


# 2. Map columns to the 'str' type
num_map = ({num: 'Int64' for num in num_cols})

# 3. Apply the conversion
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls.astype(num_map) 
enhanced_data_with_no_nulls.dtypes

Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG                 Int64
FTAG                 Int64
FTR                 object
HTHG                 Int64
HTAG                 Int64
HTR                 object
Referee             object
HS                   Int64
AS                 float64
HST                  Int64
AST                  Int64
HF                   Int64
AF                   Int64
HC                   Int64
AC                   Int64
HY                   Int64
AY                   Int64
HR                   Int64
AR                   Int64
Month               object
Year                 Int64
Day                 object
dtype: object

In [42]:
enhanced_data_with_no_nulls.head(6)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Month,Year,Day
0,2024-08-16,Man United,Fulham,1,0,H,0,0,D,R Jones,...,10,7,8,2,3,0,0,August,2024,Friday
1,2024-08-17,Ipswich,Liverpool,0,2,A,0,0,D,T Robinson,...,18,2,10,3,1,0,0,August,2024,Saturday
2,2024-08-17,Arsenal,Wolves,2,0,H,1,0,H,J Gillett,...,14,8,2,2,2,0,0,August,2024,Saturday
3,2024-08-17,Everton,Brighton,0,3,A,0,1,A,S Hooper,...,8,1,5,1,1,1,0,August,2024,Saturday
4,2024-08-17,Newcastle,Southampton,1,0,H,1,0,H,C Pawson,...,16,3,12,2,4,1,0,August,2024,Saturday
5,2024-08-17,Nott'm Forest,Bournemouth,1,1,D,1,0,H,M Oliver,...,8,2,6,1,3,0,0,August,2024,Saturday


In [43]:
enhanced_data_with_no_nulls.shape

(6841, 25)

In [44]:
import numpy as np
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls.replace(r'^\s*$', np.nan, regex=True)
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls.dropna(how='all')
enhanced_data_with_no_nulls.shape

(6841, 25)

In [45]:
# Only keep rows where 'HomeTeam' (or another core column) actually has a value
enhanced_data_with_no_nulls = enhanced_data_with_no_nulls[enhanced_data_with_no_nulls['HomeTeam'].str.strip().astype(bool)]
enhanced_data_with_no_nulls.shape

(6841, 25)

In [46]:
#enhanced_data_with_no_nulls.isnull()
null = enhanced_data_with_no_nulls.columns[enhanced_data_with_no_nulls.isnull().any()] 
null_counts  =enhanced_data_with_no_nulls.isnull().sum()
null_counts

Date        1
HomeTeam    0
AwayTeam    0
FTHG        1
FTAG        1
FTR         0
HTHG        1
HTAG        1
HTR         0
Referee     0
HS          1
AS          1
HST         1
AST         1
HF          1
AF          1
HC          1
AC          1
HY          1
AY          1
HR          1
AR          1
Month       0
Year        1
Day         0
dtype: int64

In [47]:
# ===== Find rows with ANY missing values ===== #
rows_with_nulls = enhanced_data_with_no_nulls[enhanced_data_with_no_nulls.isnull().any(axis=1)]
print(f"Number of rows with at least one missing value: {len(rows_with_nulls)}")
print(f"\nFirst 10 rows with missing values:")
rows_with_nulls.head(10)


Number of rows with at least one missing value: 1

First 10 rows with missing values:


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Month,Year,Day
4180,NaT,nan,nan,<NA>,<NA>,nan,<NA>,<NA>,nan,nan,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,nan,<NA>,nan


In [48]:
# ===== METHOD 2: Drop rows missing in SPECIFIC columns only ===== #
# Example: Only drop if missing in critical columns
critical_cols = ['HomeTeam', 'AwayTeam', 'Date', 'FTHG', 'FTAG']
enhanced_data_clean_v2 = enhanced_data_with_no_nulls.dropna(subset=critical_cols)
print(f"Original shape: {enhanced_data_with_no_nulls.shape}")
print(f"After dropping rows missing in {critical_cols}: {enhanced_data_clean_v2.shape}")
print(f"Rows removed: {enhanced_data_with_no_nulls.shape[0] - enhanced_data_clean_v2.shape[0]}")
print(f"Original shape: {enhanced_data_clean_v2.shape}")


Original shape: (6841, 25)
After dropping rows missing in ['HomeTeam', 'AwayTeam', 'Date', 'FTHG', 'FTAG']: (6840, 25)
Rows removed: 1
Original shape: (6840, 25)


In [49]:
rows_with_nulls = enhanced_data_clean_v2[enhanced_data_clean_v2.isnull().any(axis=1)]
print(f"Number of rows with at least one missing value: {len(rows_with_nulls)}")
print(f"\nFirst 10 rows with missing values:")
rows_with_nulls.head(10)



Number of rows with at least one missing value: 0

First 10 rows with missing values:


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Month,Year,Day


In [50]:
import psycopg2
from dotenv import load_dotenv
import os
# Get the connection string from the environment variable

load_dotenv()  # Loads .env file

db_name = os.getenv('NEON_DB_NAME')
db_role = os.getenv('NEON_DB_ROLE')
db_string = os.getenv('NEON_CONNECTION_STRING')


In [51]:
try:
    with psycopg2.connect(db_string) as conn:
        print("✅ Connection established")
        
        with conn.cursor() as cur:
            # Check if table exists
            cur.execute("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'epl_data');")
            table_exists = cur.fetchone()[0]
            
            if table_exists:
                print("✅ Table 'epl_data' already exists")
            else:
                # Create table
                cur.execute("""
                    CREATE TABLE epl_data(
                        id SERIAL PRIMARY KEY,
                        Date DATE NULL,
                        HomeTeam VARCHAR(255) NOT NULL,
                        AwayTeam VARCHAR(255) NOT NULL,
                        FTHG SMALLINT,
                        FTAG SMALLINT,
                        FTR CHAR(1),
                        HTHG SMALLINT,
                        HTAG SMALLINT,
                        Referee VARCHAR(255),
                        HomeShots SMALLINT,
                        AwayShots SMALLINT,
                        HST SMALLINT,
                        AST SMALLINT,
                        HF SMALLINT,
                        AF SMALLINT,
                        HC SMALLINT,
                        AC SMALLINT,
                        HY SMALLINT,
                        AY SMALLINT,
                        HR SMALLINT,
                        AR SMALLINT,
                        Month VARCHAR(255),
                        Year SMALLINT,
                        Day VARCHAR(255)
                    );
                """)
                print("✅ Table created successfully in Neon")
        
        conn.commit()
        
except Exception as e:
    print("❌ Connection failed")
    print(e)

✅ Connection established
✅ Table 'epl_data' already exists


In [52]:
## Adding the data to the database

# Create a proper independent copy and reset index
enhanced_data_clean_v2 = enhanced_data_clean_v2.copy()
enhanced_data_clean_v2 = enhanced_data_clean_v2.reset_index(drop=True)

print(f"✅ Data prepared for insertion: {enhanced_data_clean_v2.shape}")
print(enhanced_data_clean_v2.head())


✅ Data prepared for insertion: (6840, 25)
        Date    HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0 2024-08-16  Man United       Fulham     1     0   H     0     0   D   
1 2024-08-17     Ipswich    Liverpool     0     2   A     0     0   D   
2 2024-08-17     Arsenal       Wolves     2     0   H     1     0   H   
3 2024-08-17     Everton     Brighton     0     3   A     0     1   A   
4 2024-08-17   Newcastle  Southampton     1     0   H     1     0   H   

      Referee  ...  AF  HC  AC  HY  AY  HR  AR   Month  Year       Day  
0     R Jones  ...  10   7   8   2   3   0   0  August  2024    Friday  
1  T Robinson  ...  18   2  10   3   1   0   0  August  2024  Saturday  
2   J Gillett  ...  14   8   2   2   2   0   0  August  2024  Saturday  
3    S Hooper  ...   8   1   5   1   1   1   0  August  2024  Saturday  
4    C Pawson  ...  16   3  12   2   4   1   0  August  2024  Saturday  

[5 rows x 25 columns]


In [53]:
# Rename columns
enhanced_data_clean_v2.rename(columns={
    'HS': 'HomeShots',
    'AS': 'AwayShots'
}, inplace=True)
enhanced_data_clean_v2.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HomeShots', 'AwayShots', 'HST', 'AST', 'HF', 'AF',
       'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Month', 'Year', 'Day'],
      dtype='object')

In [54]:

# Reorder columns to match INSERT statement
columns_order = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'Referee', 
                 'HomeShots', 'AwayShots', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Month', 'Year', 'Day']

# Only keep columns that exist in the dataframe
columns_order = [col for col in columns_order if col in enhanced_data_clean_v2.columns]

enhanced_data_clean_v2 = enhanced_data_clean_v2[columns_order]

print(f"✅ Columns ordered: {list(enhanced_data_clean_v2.columns)}")
print(f"✅ Data shape: {enhanced_data_clean_v2.shape}")

# Convert to tuples
data_to_insert = [tuple(row) for row in enhanced_data_clean_v2.values]
print(f"✅ Created {len(data_to_insert)} tuples for insertion")


✅ Columns ordered: ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'Referee', 'HomeShots', 'AwayShots', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Month', 'Year', 'Day']
✅ Data shape: (6840, 24)
✅ Created 6840 tuples for insertion


In [55]:
# Establish the connection and insert data
conn = None
try:
    conn = psycopg2.connect(db_string)
    cur = conn.cursor()

    # Get actual columns from the dataframe
    actual_columns = list(enhanced_data_clean_v2.columns)
    num_columns = len(actual_columns)
    
    # Create placeholders dynamically based on actual columns
    placeholders = ', '.join(['%s'] * num_columns)
    # Convert column names to lowercase to match table schema (PostgreSQL stores unquoted names as lowercase)
    columns_str = ', '.join([col.lower() for col in actual_columns])
    
    # Build INSERT statement dynamically
    insert_query = f"INSERT INTO epl_data ({columns_str}) VALUES ({placeholders})"
    
    print(f"Inserting {num_columns} columns:")
    print(f"Columns: {actual_columns}")
    print(f"Query: {insert_query[:100]}...")

    cur.executemany(insert_query, data_to_insert)
    print(f"✅ Inserted {cur.rowcount} rows into 'epl_data' table")

    # Commit the changes to the database
    conn.commit()
    print("✅ Changes committed successfully")

except Exception as e:
    print(f"❌ Error: {e}")
    if conn:
        conn.rollback()
        print("Transaction rolled back")
finally:
    if conn:
        cur.close()
        conn.close()
        print("✅ Database connection closed")


Inserting 24 columns:
Columns: ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'Referee', 'HomeShots', 'AwayShots', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'Month', 'Year', 'Day']
Query: INSERT INTO epl_data (date, hometeam, awayteam, fthg, ftag, ftr, hthg, htag, referee, homeshots, awa...
✅ Inserted 6840 rows into 'epl_data' table
✅ Changes committed successfully
✅ Database connection closed


In [ ]:


    def filter_data_by_non_betting_features(self):
        print("Data Cleaning: Step 1: Filtering feature catalog to exclude Betting_odd=True features.......")
        logging.info("Entering the Filtering out the non betting features module.....")
        
        # Load the CSV files first
        feature_catalog_df = pd.read_csv(self.ingestion_config.feature_catalog_csv_file)
        enhanced_data_df = pd.read_csv(self.ingestion_config.ingested_csv_file,low_memory=False)
        
        # Now filter
        # Had to Convert the False to avoid the boolean confusion brought about by Singleton Comparison
        is_not_betting_odd = feature_catalog_df['Betting_odd'].astype(str).str.lower().isin(['false'])
        feature_catalog_filtered = feature_catalog_df[is_not_betting_odd].copy()
        feature_names_to_keep = feature_catalog_filtered['Feature_name'].unique()
        enhanced_data = enhanced_data_df.loc[:, enhanced_data_df.columns.isin(feature_names_to_keep)].copy()


        columns_retained = enhanced_data.columns
        logging.info(f" Here are the retained columns after removing betting odds : {columns_retained}")
        logging.info(f"The new number of columns are {enhanced_data.shape[1]} and the new number of the rows are {enhanced_data.shape[0]}" )
        return enhanced_data
    

    def removing_columns_missing_significant_chunks(self,enhanced_data):
        print("Data Cleaning:Step 2: Removing columns missing significant chunks on it.......")
        try:
            total = enhanced_data.isnull().sum().sort_values(ascending=False)
            missing_percent = (enhanced_data.isnull().sum() / len(enhanced_data)) * 100
            missing_data = pd.concat([total,missing_percent],axis=1, keys=['Total','Percent'])
            logging.info(f"These are the columns that are missing{missing_data.head(20)}")
            #Deleting the columns
            columns_to_drop = missing_percent[missing_percent > 10].index
            enhanced_data_with_no_nulls = enhanced_data.drop(columns=columns_to_drop).copy()
            print(f"Dropped {len(columns_to_drop)} columns.")
            logging.info(f"Dropped {len(columns_to_drop)} columns.")
            print(f"Dropped {columns_to_drop} columns.")
            logging.info(f"Dropped {columns_to_drop} columns.")
            logging.info(f"{enhanced_data_with_no_nulls.head()}")

            #Cleaning up remaining rows
            critical_cols = ['HomeTeam', 'AwayTeam', 'Date', 'FTHG', 'FTAG']
            enhanced_data_with_no_nulls_v2 = enhanced_data_with_no_nulls.dropna(subset=critical_cols)
            logging.info(f"Original shape: {enhanced_data_with_no_nulls.shape}")
            logging.info(f"After dropping rows missing in {critical_cols}: {enhanced_data_with_no_nulls_v2.shape}")
            logging.info(f"Rows removed: {enhanced_data_with_no_nulls.shape[0] - enhanced_data_with_no_nulls_v2.shape[0]}")
            logging.info(f"New shape: {enhanced_data_with_no_nulls_v2.shape}")

            #Just checking whether they are any "Nan" rows left
            rows_with_nulls = enhanced_data_with_no_nulls_v2[enhanced_data_with_no_nulls_v2.isnull().any(axis=1)]
            logging.info(f"Number of rows with at least one missing value: {len(rows_with_nulls)}")
            logging.info(f"\nFirst 10 rows with missing values:")
            logging.info(rows_with_nulls.head(10))




        except Exception as e:
            raise CustomException(e,sys) from e
        print("Data Cleaning:Step 2: Removing columns missing significant chunks on itâœ…")
        return enhanced_data_with_no_nulls_v2
    

    def declare_dtypes_early(self,enhanced_data_with_no_nulls_v2):
        print("Data Cleaning:Step 3: Enforcing data types.......")
        try:
            logging.info("Enforcing and Augmenting Date Data types")
            enhanced_data_with_no_nulls_v2['Date'] = pd.to_datetime(enhanced_data_with_no_nulls_v2['Date'],format = 'mixed')
            enhanced_data_with_no_nulls_v2['Month'] = enhanced_data_with_no_nulls_v2['Date'].dt.strftime('%B')
            enhanced_data_with_no_nulls_v2['Year'] = enhanced_data_with_no_nulls_v2['Date'].dt.year
            enhanced_data_with_no_nulls_v2['Day'] = enhanced_data_with_no_nulls_v2['Date'].dt.strftime('%A')
            logging.info(f"Augmented date types : {enhanced_data_with_no_nulls_v2.sample(10)}")

            # == Fixing String Columns == #
            logging.info("Enforcing string columns to avoid edge cases....")
            string_cols = ['HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'Referee', 'Month', 'Day'] 
            str_map = ({col: str for col in string_cols})
            enhanced_data_with_no_nulls_v2 = enhanced_data_with_no_nulls_v2.astype(str_map) 
            logging.info(f"Here are the data types: {enhanced_data_with_no_nulls_v2.dtypes}")

            # == Fixing Numeric Columns == #
            logging.info("Enforcing numeric columns......")
            num_cols = ['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'HS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR','Year']
            num_map = ({num: 'Int64' for num in num_cols})
            enhanced_data_with_no_nulls_v2 = enhanced_data_with_no_nulls_v2.astype(num_map)
            logging.info(f"Here are the data types: {enhanced_data_with_no_nulls_v2.dtypes}")
        except Exception as e:
            raise CustomException(e,sys) from e
        print("Data Cleaning:Step 3: Enforcing data types.......âœ…")
        logging.info("Data Cleaning:Step 3: Enforcing data types.......DONEâœ…")
        return enhanced_data_with_no_nulls_v2
    